# PostgreSQL Connection Example

This notebook demonstrates how to connect to a PostgreSQL database from Jupyter and perform basic operations.

## 1. Install Required Libraries (if needed)
If you haven't already, install the required packages:

In [1]:
# Uncomment and run if you need to install these packages
!pip install sqlalchemy psycopg2-binary pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 150.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 144.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 153.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.1/579.1 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 173.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [pandas]2m6/7 [pandas]emy]nary]


## 2. Import Libraries

In [2]:
import pandas as pd
from sqlalchemy import create_engine, text
import os

## 3. Configure Database Connection

Use the connection details from your Docker Compose setup:

In [3]:
# Database connection parameters
DB_USER = 'postgres'
DB_PASSWORD = 'postgres'
DB_HOST = 'localhost'  # Use 'postgres' if running within the Docker network, 'localhost' if connecting from outside
DB_PORT = '5432'
DB_NAME = 'weather_data'

# Create connection string
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create SQLAlchemy engine
engine = create_engine(connection_string)

## 4. Test Connection

In [4]:
# Test the connection
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1"))
        print("Connection successful!")
except Exception as e:
    print(f"Error connecting to the database: {e}")

Connection successful!


## 5. Create a Sample Table

In [5]:
# Create a sample weather stations table
create_table_query = """
CREATE TABLE IF NOT EXISTS weather_stations (
    station_id VARCHAR(20) PRIMARY KEY,
    station_name VARCHAR(100),
    latitude FLOAT,
    longitude FLOAT,
    elevation FLOAT,
    country VARCHAR(50),
    state VARCHAR(50)
);
"""

try:
    with engine.connect() as connection:
        connection.execute(text(create_table_query))
        connection.commit()
        print("Table created successfully!")
except Exception as e:
    print(f"Error creating table: {e}")

Table created successfully!


## 6. Insert Sample Data

In [6]:
# Sample data as a pandas DataFrame
sample_stations = pd.DataFrame([
    {'station_id': 'USW00094728', 'station_name': 'NEW YORK CENTRAL PARK', 'latitude': 40.7789, 'longitude': -73.9692, 'elevation': 42.7, 'country': 'US', 'state': 'NY'},
    {'station_id': 'USW00023174', 'station_name': 'CHICAGO OHARE INTL AP', 'latitude': 41.9950, 'longitude': -87.9336, 'elevation': 201.8, 'country': 'US', 'state': 'IL'},
    {'station_id': 'USW00023234', 'station_name': 'WASHINGTON REAGAN AP', 'latitude': 38.8572, 'longitude': -77.0372, 'elevation': 3.0, 'country': 'US', 'state': 'DC'},
    {'station_id': 'USW00023188', 'station_name': 'SAN FRANCISCO INTL AP', 'latitude': 37.6197, 'longitude': -122.3647, 'elevation': 2.4, 'country': 'US', 'state': 'CA'}
])

# Write the DataFrame to the PostgreSQL table
try:
    sample_stations.to_sql('weather_stations', engine, if_exists='append', index=False)
    print("Sample data inserted successfully!")
except Exception as e:
    print(f"Error inserting data: {e}")

Sample data inserted successfully!


## 7. Query the Data

In [7]:
# Query the data
query = "SELECT * FROM weather_stations;"

try:
    stations_df = pd.read_sql(query, engine)
    display(stations_df)
except Exception as e:
    print(f"Error querying data: {e}")

,station_id,station_name,latitude,longitude,elevation,country,state
0,USW00094728,NEW YORK CENTRAL PARK,40.7789,-73.9692,42.7,US,NY
1,USW00023174,CHICAGO OHARE INTL AP,41.9950,-87.9336,201.8,US,IL
2,USW00023234,WASHINGTON REAGAN AP,38.8572,-77.0372,3.0,US,DC
3,USW00023188,SAN FRANCISCO INTL AP,37.6197,-122.3647,2.4,US,CA


## 8. Create a Sample Weather Readings Table

In [8]:
# Create a sample weather readings table
create_readings_table_query = """
CREATE TABLE IF NOT EXISTS weather_readings (
    id SERIAL PRIMARY KEY,
    station_id VARCHAR(20) REFERENCES weather_stations(station_id),
    date DATE,
    element VARCHAR(10),
    value FLOAT,
    quality_flag VARCHAR(1)
);
"""

try:
    with engine.connect() as connection:
        connection.execute(text(create_readings_table_query))
        connection.commit()
        print("Weather readings table created successfully!")
except Exception as e:
    print(f"Error creating weather readings table: {e}")

Weather readings table created successfully!


## 9. Insert Sample Weather Readings

In [9]:
# Sample weather readings data
from datetime import date, timedelta
import random

# Generate some sample data for the past week
readings = []
elements = ['TMAX', 'TMIN', 'PRCP']
today = date.today()

for station_id in sample_stations['station_id']:
    for i in range(7):  # Past 7 days
        for element in elements:
            current_date = today - timedelta(days=i)
            
            # Generate random values based on element type
            if element == 'TMAX':
                value = round(random.uniform(15, 35), 1)  # Temperature in Celsius
            elif element == 'TMIN':
                value = round(random.uniform(5, 20), 1)  # Temperature in Celsius
            else:  # PRCP
                value = round(random.uniform(0, 25), 1)  # Precipitation in mm
                
            readings.append({
                'station_id': station_id,
                'date': current_date,
                'element': element,
                'value': value,
                'quality_flag': ''
            })

# Convert to DataFrame
readings_df = pd.DataFrame(readings)

# Insert into database
try:
    readings_df.to_sql('weather_readings', engine, if_exists='append', index=False)
    print(f"Inserted {len(readings_df)} sample weather readings successfully!")
except Exception as e:
    print(f"Error inserting weather readings: {e}")

Inserted 84 sample weather readings successfully!


## 10. Query Weather Readings with Joins

In [10]:
# Query to get weather readings with station information
join_query = """
SELECT 
    r.date, 
    s.station_name, 
    s.state,
    r.element, 
    r.value
FROM 
    weather_readings r
JOIN 
    weather_stations s ON r.station_id = s.station_id
WHERE 
    r.element = 'TMAX'
ORDER BY 
    r.date DESC, s.station_name
LIMIT 20;
"""

try:
    result_df = pd.read_sql(join_query, engine)
    display(result_df)
except Exception as e:
    print(f"Error executing join query: {e}")

,date,station_name,state,element,value
0,2025-04-27,CHICAGO OHARE INTL AP,IL,TMAX,21.6
1,2025-04-27,NEW YORK CENTRAL PARK,NY,TMAX,30.3
2,2025-04-27,SAN FRANCISCO INTL AP,CA,TMAX,16.4
3,2025-04-27,WASHINGTON REAGAN AP,DC,TMAX,30.3
4,2025-04-26,CHICAGO OHARE INTL AP,IL,TMAX,27.2
5,2025-04-26,NEW YORK CENTRAL PARK,NY,TMAX,27.5
6,2025-04-26,SAN FRANCISCO INTL AP,CA,TMAX,30.4
7,2025-04-26,WASHINGTON REAGAN AP,DC,TMAX,29.2
8,2025-04-25,CHICAGO OHARE INTL AP,IL,TMAX,16.9
9,2025-04-25,NEW YORK CENTRAL PARK,NY,TMAX,25.0


## 11. Basic Data Analysis and Visualization

In [11]:
# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Query for temperature data
temp_query = """
SELECT 
    r.date, 
    s.station_name, 
    r.element, 
    r.value
FROM 
    weather_readings r
JOIN 
    weather_stations s ON r.station_id = s.station_id
WHERE 
    r.element IN ('TMAX', 'TMIN')
ORDER BY 
    r.date, s.station_name, r.element;
"""

temp_df = pd.read_sql(temp_query, engine)

# Pivot the data for easier plotting
pivot_df = temp_df.pivot_table(
    index=['date', 'station_name'],
    columns='element',
    values='value'
).reset_index()

# Set plot style
plt.figure(figsize=(12, 8))
sns.set_style('whitegrid')

# Plot for each station
for station in pivot_df['station_name'].unique():
    station_data = pivot_df[pivot_df['station_name'] == station]
    plt.plot(station_data['date'], station_data['TMAX'], 'o-', label=f"{station} (Max)")
    plt.plot(station_data['date'], station_data['TMIN'], 'o--', label=f"{station} (Min)")

plt.title('Temperature Trends by Station', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Temperature (°C)', fontsize=12)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

## 12. Close Connection

It's good practice to dispose of the engine when you're done with it:

In [12]:
# Dispose of the engine
engine.dispose()
print("Connection closed.")

Connection closed.
